In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Brain Tumor Detection**

In [ ]:
# Data Managment
import pandas as pd
import numpy as np
import cv2 as cv 
import matplotlib.pyplot as plt
import os
import shutil

# CV model 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import (Conv2DTranspose,
                                    Concatenate,
                                    MaxPooling2D,
                                    Dense,
                                    Flatten,
                                    Dropout,
                                    GlobalAveragePooling2D,
                                    Conv2D,
                                    Input,
                                    MaxPooling2D)

from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv(r"/kaggle/input/brian-tumor-dataset/metadata_rgb_only.csv")
df.head()

## **Exploring and Preparing The Data**

In [ ]:
def read_img(image_path, size):
    # Read image
    img = cv.imread(image_path)
    
    if img is None:
        raise ValueError(f"Could not read image: {image_path}")
    
    # Resize to (512, 512)
    img_resized = cv.resize(img, (size, size))
    
    # Convert to grayscale
    img_gray = cv.cvtColor(img_resized, cv.COLOR_BGR2GRAY)
    
    return img_gray

def read_dataset(dataset_path):
    X = []
    y = []
    categories = ["Brain Tumor", "Healthy"]
    
    for category in categories:
        category_path = os.path.join(dataset_path, category)
        label = 1 if category == "Brain Tumor" else 0  # Assign labels (1 for tumor, 0 for healthy)
        
        for filename in os.listdir(category_path):
            file_path = os.path.join(category_path, filename)
            try:
                img = read_img(file_path, 512)
                X.append(img)
                y.append(label)
            except Exception as e:
                print(f"Error processing {file_path}: {e}")
    
    return X, y

X, y = read_dataset("/kaggle/input/brian-tumor-dataset/Brain Tumor Data Set/Brain Tumor Data Set")

In [ ]:
plt.imshow(X[5]);
plt.axis("off");

In [ ]:
def prepare_dataset(dataset_path, output_dir, validation_split=0.22, test_split=0.1):
    """
    Organizes the dataset into train, validation, and test folders.
    """
    classes = os.listdir(dataset_path)  # Get class names (Tumor / Normal)
    
    for class_name in classes:
        class_path = os.path.join(dataset_path, class_name)
        images = os.listdir(class_path)
        
        # Split dataset into train, validation, and test
        train_images, temp_images = train_test_split(images, test_size=(validation_split + test_split), stratify=None)
        val_images, test_images = train_test_split(temp_images, test_size=test_split / (validation_split + test_split), stratify=None)
        
        for split, split_images in zip(["train", "val", "test"], [train_images, val_images, test_images]):
            split_path = os.path.join(output_dir, split, class_name)
            os.makedirs(split_path, exist_ok=True)
            
            for img in split_images:
                src_path = os.path.join(class_path, img)
                dst_path = os.path.join(split_path, img)
                shutil.copy(src_path, dst_path)
    
    print("Dataset prepared successfully!")

# Paths
dataset_path = "/kaggle/input/brian-tumor-dataset/Brain Tumor Data Set/Brain Tumor Data Set"
output_dir = "/kaggle/working/processed_dataset"

# Organize dataset into train, val, and test
prepare_dataset(dataset_path, output_dir)

In [ ]:
def create_data_generators(data_dir, batch_size=32):
    """
    Creates ImageDataGenerators for training, validation, and testing,
    applying ResNet-specific preprocessing.
    """
    datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
    
    train_generator = datagen.flow_from_directory(
        os.path.join(data_dir, "train"),
        target_size=(224, 224),  # ResNet standard input size
        batch_size=batch_size,
        color_mode='rgb',
        class_mode='binary'
    )
    
    validation_generator = datagen.flow_from_directory(
        os.path.join(data_dir, "val"),
        target_size=(224, 224),
        batch_size=batch_size,
        color_mode='rgb',
        class_mode='binary'
    )
    
    test_generator = datagen.flow_from_directory(
        os.path.join(data_dir, "test"),
        target_size=(224, 224),
        batch_size=batch_size,
        color_mode='rgb',
        class_mode='binary',
        shuffle=False  # Keep order for test evaluation
    )
    
    return train_generator, validation_generator, test_generator

# Example usage:
train_gen, valid_gen, test_gen = create_data_generators(output_dir)

In [ ]:
def create_data_generators(data_dir, batch_size=32):
    """
    Creates ImageDataGenerators for training, validation, and testing.
    """
    datagen = ImageDataGenerator(rescale=1./255)
    
    train_generator = datagen.flow_from_directory(
        os.path.join(data_dir, "train"),
        target_size=(512, 512),
        batch_size=batch_size,
        color_mode='grayscale',
        class_mode='binary'
    )
    
    validation_generator = datagen.flow_from_directory(
        os.path.join(data_dir, "val"),
        target_size=(512, 512),
        batch_size=batch_size,
        color_mode='grayscale',
        class_mode='binary'
    )
    
    test_generator = datagen.flow_from_directory(
        os.path.join(data_dir, "test"),
        target_size=(512, 512),
        batch_size=batch_size,
        color_mode='grayscale',
        class_mode='binary'
    )
    
    return train_generator, validation_generator, test_generator

# Create data generators
train_gen, valid_gen, test_gen = create_data_generators(output_dir)

### **Model Teaining and Evaluation**

In [ ]:
# Load base model with pretrained weights, excluding the top layer
base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=Input(shape=(224, 224, 3)))

# Freeze base model layers (optional, good for transfer learning)
base_model.trainable = False

# Custom classification head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.2)(x)
x = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=x)

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy', 'precision', 'recall'])

model.summary()

In [ ]:
model.fit(
    train_gen,
    validation_data=valid_gen,
    epochs=10,
    verbose=1
)

In [ ]:
loss, acc, prec, rec = model.evaluate(test_gen)
print(f"Loss: {loss}")
print(f"Accuracy: {acc}")
print(f"Precision: {prec}")
print(f"Recall: {rec}")

In [ ]:
model_save_path = "/kaggle/working/model/resnet_brain_model.h5"
model.save(model_save_path)

print(f"Model saved successfully at {model_save_path}")

In [ ]:
import kagglehub

LOCAL_MODEL_DIR = '/kaggle/working/model'  # Directory where your model files are saved
MODEL_SLUG = 'brain-tumor-resnet'  # Use the model slug from your Kaggle model page
VARIATION_SLUG = 'v2' 


kagglehub.model_upload(
    handle=f"khalednabawi/{MODEL_SLUG}/keras/{VARIATION_SLUG}",
    local_model_dir=LOCAL_MODEL_DIR,
    version_notes="Upload - Brain Tumor Resnet"
)

In [ ]:
cnn_model = Sequential([
    Input(shape=(512, 512, 1)),
    Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'),
    Conv2D(filters=32, kernel_size=3, activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    
    Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'),
    Conv2D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    
    Conv2D(filters=128, kernel_size=3, padding='same', activation='relu'),
    Conv2D(filters=128, kernel_size=3, activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    
    Conv2D(filters=256, kernel_size=3, padding='same', activation='relu'),
    Conv2D(filters=256, kernel_size=3, activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    
    Dropout(0.25),
    Flatten(),
    Dense(units=512, activation='relu'),
    Dropout(0.4),
    
    Dense(1, activation='sigmoid')
])

cnn_model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy', 'precision', 'recall'])

In [ ]:
cnn_model.fit(
    train_gen,
    validation_data=valid_gen,
    epochs=10,
    verbose=1
)

In [ ]:
cnn_model.evaluate(test_gen)

In [ ]:
model_save_path = "/kaggle/working/model/cnn_brain_tumor_model.h5"
cnn_model.save(model_save_path)  # Saves the model in TensorFlow SavedModel format

print(f"Model saved successfully at {model_save_path}")

In [ ]:
!pip install --upgrade kagglehub

In [ ]:
import kagglehub

LOCAL_MODEL_DIR = '/kaggle/working/model'  # Directory where your model files are saved
MODEL_SLUG = 'brain-tumor-cnn'  # Use the model slug from your Kaggle model page
VARIATION_SLUG = 'v1' 


kagglehub.model_upload(
    handle=f"khalednabawi/{MODEL_SLUG}/keras/{VARIATION_SLUG}",
    local_model_dir=LOCAL_MODEL_DIR,
    version_notes="Upload - Brain Tumor CNN"
)

In [ ]:
import json

model_metadata = {
    "title": "Brain Tumor CNN Model",
    "slug": "brain-tumor-cnn",  # ✅ Add a unique slug for the model
    "subtitle": "A CNN model trained to detect brain tumors",
    "description": "This model is trained on a brain tumor dataset using TensorFlow and can be used for classification.",
    "isPrivate": True,  # Set to False if you want it public
    "licenses": [{"name": "CC0-1.0"}],  
    "ownerSlug": kaggle_username
}

with open("/kaggle/working/model-metadata.json", "w") as f:
    json.dump(model_metadata, f)

In [ ]:
import os
import cv2
import numpy as np
import shutil
from sklearn.model_selection import train_test_split

def create_mask(image_path):
    """
    Generates a binary mask from the input image.
    """
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Read in grayscale
    _, mask = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)  # Convert to binary mask
    return mask

def prepare_masks(dataset_path, output_dir, validation_split=0.22, test_split=0.1):
    """
    Generates and organizes masks into train, validation, and test folders.
    """
    classes = os.listdir(dataset_path)  # Get class names (Tumor / Normal)
    
    for class_name in classes:
        class_path = os.path.join(dataset_path, class_name)
        images = sorted(os.listdir(class_path))  # Sort to keep consistency
        
        # Split dataset
        train_images, temp_images = train_test_split(images, test_size=(validation_split + test_split), random_state=42)
        val_images, test_images = train_test_split(temp_images, test_size=test_split / (validation_split + test_split), random_state=42)
        
        for split, split_images in zip(["train", "val", "test"], [train_images, val_images, test_images]):
            mask_split_path = os.path.join(output_dir, split, class_name)
            os.makedirs(mask_split_path, exist_ok=True)
            
            for img in split_images:
                img_path = os.path.join(class_path, img)
                mask = create_mask(img_path)  # Generate mask
                
                mask_filename = os.path.splitext(img)[0] + "_mask.png"
                mask_path = os.path.join(mask_split_path, mask_filename)
                
                cv2.imwrite(mask_path, mask)  # Save mask
                
    print("Masks generated and organized successfully!")

# Paths
dataset_path = "/kaggle/input/brian-tumor-dataset/Brain Tumor Data Set/Brain Tumor Data Set"
output_dir = "/kaggle/working/masks_dataset"

# Generate and save masks
prepare_masks(dataset_path, output_dir)

In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def prepare_dataset(dataset_path, mask_path, output_dir, validation_split=0.22, test_split=0.1):
    """
    Organizes the dataset into train, validation, and test folders for images and masks.
    """
    classes = os.listdir(dataset_path)  # Get class names (Tumor / Normal)
    
    for class_name in classes:
        class_image_path = os.path.join(dataset_path, class_name)
        class_mask_path = os.path.join(mask_path, class_name)
        images = os.listdir(class_image_path)
        
        # Split dataset into train, validation, and test
        train_images, temp_images = train_test_split(images, test_size=(validation_split + test_split), stratify=None)
        val_images, test_images = train_test_split(temp_images, test_size=test_split / (validation_split + test_split), stratify=None)
        
        for split, split_images in zip(["train", "val", "test"], [train_images, val_images, test_images]):
            split_image_path = os.path.join(output_dir, split, "images", class_name)
            split_mask_path = os.path.join(output_dir, split, "masks", class_name)
            os.makedirs(split_image_path, exist_ok=True)
            os.makedirs(split_mask_path, exist_ok=True)
            
            for img in split_images:
                img_src_path = os.path.join(class_image_path, img)
                img_dst_path = os.path.join(split_image_path, img)
                shutil.copy(img_src_path, img_dst_path)
                
                mask_src_path = os.path.join(class_mask_path, img)
                mask_dst_path = os.path.join(split_mask_path, img)
                if os.path.exists(mask_src_path):  # Ensure the mask exists before copying
                    shutil.copy(mask_src_path, mask_dst_path)
    
    print("Dataset and masks prepared successfully!")

def create_data_generators(data_dir, batch_size=32):
    """
    Creates ImageDataGenerators for training, validation, and testing.
    """
    datagen = ImageDataGenerator(rescale=1./255)
    
    train_generator = datagen.flow_from_directory(
        os.path.join(data_dir, "train", "images"),
        target_size=(512, 512),
        batch_size=batch_size,
        color_mode='grayscale',
        class_mode='binary'
    )
    
    validation_generator = datagen.flow_from_directory(
        os.path.join(data_dir, "val", "images"),
        target_size=(512, 512),
        batch_size=batch_size,
        color_mode='grayscale',
        class_mode='binary'
    )
    
    test_generator = datagen.flow_from_directory(
        os.path.join(data_dir, "test", "images"),
        target_size=(512, 512),
        batch_size=batch_size,
        color_mode='grayscale',
        class_mode='binary'
    )
    
    return train_generator, validation_generator, test_generator

# Paths
dataset_path = "/kaggle/input/brian-tumor-dataset/Brain Tumor Data Set/Brain Tumor Data Set"
mask_path = "/kaggle/input/brian-tumor-dataset/Brain Tumor Data Set/Brain Tumor Masks"
output_dir = "/kaggle/working/processed_dataset"

# Organize dataset and masks into train, val, and test
prepare_dataset(dataset_path, mask_path, output_dir)

# Create data generators
train_gen, valid_gen, test_gen = create_data_generators(output_dir)

In [ ]:
# np.array(y_mask).shape
plt.imshow(y_mask[0]);
plt.axis("off");

In [ ]:
from sklearn.model_selection import train_test_split
import tensorflow as tf

# Split data: 70% train, 20% validation, 10% test
X_train, X_temp, y_mask_train, y_mask_temp = train_test_split(X, y_mask, test_size=0.3, random_state=42)
X_val, X_test, y_mask_val, y_mask_test = train_test_split(X_temp, y_mask_temp, test_size=0.33, random_state=42)  # 33% of 30% = ~10%

# Convert lists to NumPy arrays for better performance
X_train = np.array(X_train)
X_val = np.array(X_val)
X_test = np.array(X_test)
y_mask_train = np.array(y_mask_train)
y_mask_val = np.array(y_mask_val)
y_mask_test = np.array(y_mask_test)

# Convert NumPy arrays to TensorFlow tensors
X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
X_val = tf.convert_to_tensor(X_val, dtype=tf.float32)
X_test = tf.convert_to_tensor(X_test, dtype=tf.float32)
y_mask_train = tf.convert_to_tensor(y_mask_train, dtype=tf.float32)
y_mask_val = tf.convert_to_tensor(y_mask_val, dtype=tf.float32)
y_mask_test = tf.convert_to_tensor(y_mask_test, dtype=tf.float32)

# Print dataset tensor shapes
print(f"Train: {X_train.shape}, Validation: {X_val.shape}, Test: {X_test.shape}")
print(f"Train Masks: {y_mask_train.shape}, Validation Masks: {y_mask_val.shape}, Test Masks: {y_mask_test.shape}")

In [ ]:
import os
import cv2 as cv
import numpy as np

# Define paths
dataset_path = "/kaggle/input/brian-tumor-dataset/Brain Tumor Data Set/Brain Tumor Data Set"
mask_base_path =  "/kaggle/working/Masks"

# Create mask directories
os.makedirs(os.path.join(mask_base_path, "Brain Tumor"), exist_ok=True)
os.makedirs(os.path.join(mask_base_path, "Healthy"), exist_ok=True)

def create_binary_mask(image_path):
    """ Reads an image, converts it to grayscale, and generates a binary mask. """
    img = cv.imread(image_path, cv.IMREAD_GRAYSCALE)  # Read as grayscale
    if img is None:
        print(f"Skipping {image_path}, could not read image.")
        return None
    _, mask = cv.threshold(img, 127, 255, cv.THRESH_BINARY)  # Convert to binary mask
    return mask

# Process images
categories = ["Brain Tumor", "Healthy"]
for category in categories:
    category_path = os.path.join(dataset_path, category)
    mask_category_path = os.path.join(mask_base_path, category)

    for filename in os.listdir(category_path):
        img_path = os.path.join(category_path, filename)
        mask_path = os.path.join(mask_category_path, filename)  # Save with same name
        
        # Generate and save mask
        mask = create_binary_mask(img_path)
        if mask is not None:
            cv.imwrite(mask_path, mask)

print("✅ Mask generation complete. Masks saved in:", mask_base_path)

In [ ]:
import tensorflow as tf
import os

# Define dataset paths
image_dir = "/kaggle/input/brian-tumor-dataset/Brain Tumor Data Set/Brain Tumor Data Set"
mask_dir = "/kaggle/working/Masks"

# Image size for U-Net
IMG_SIZE = (256, 256)
BATCH_SIZE = 16

# Load image dataset as grayscale
image_dataset = tf.keras.utils.image_dataset_from_directory(
    image_dir,
    label_mode=None,  # No labels for segmentation
    color_mode="grayscale",  # Convert images to grayscale
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

# Load mask dataset as grayscale
mask_dataset = tf.keras.utils.image_dataset_from_directory(
    mask_dir,
    label_mode=None,
    color_mode="grayscale",  # Masks are also grayscale
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

# Normalize images and preprocess masks
def preprocess(image, mask):
    image = tf.cast(image, tf.float32) / 255.0  # Normalize images
    mask = tf.cast(mask, tf.float32) / 255.0  # Normalize masks
    mask = tf.where(mask > 0.5, 1.0, 0.0)  # Convert masks to binary (0 or 1)
    
    # Ensure shape is (256, 256, 1)
    image = tf.expand_dims(image, axis=-1) if image.shape[-1] != 1 else image
    mask = tf.expand_dims(mask, axis=-1) if mask.shape[-1] != 1 else mask

    return image, mask

# Combine images and masks into a dataset
dataset = tf.data.Dataset.zip((image_dataset, mask_dataset))
dataset = dataset.map(preprocess)

# Split into train, validation, and test sets
DATASET_SIZE = len(image_dataset)  # Get total dataset size
train_size = int(0.7 * DATASET_SIZE)
val_size = int(0.2 * DATASET_SIZE)
test_size = int(0.1 * DATASET_SIZE)

train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size).take(val_size)
test_dataset = dataset.skip(train_size + val_size).take(test_size)

print(f"Train: {train_size} batches, Validation: {val_size} batches, Test: {test_size} batches")

# Check dataset shape
for img, mask in train_dataset.take(1):
    print("Image shape:", img.shape)  # Expected: (batch_size, 256, 256, 1)
    print("Mask shape:", mask.shape)  # Expected: (batch_size, 256, 256, 1)

In [ ]:
def unet(sz=(256, 256, 1)):
    inputs = Input(shape=sz)

    # Downsampling
    f = 64
    layers = []
    x = inputs
    for i in range(6):
        x = Conv2D(f, 3, activation='relu', padding='same')(x)
        x = Conv2D(f, 3, activation='relu', padding='same')(x)
        layers.append(x)
        x = MaxPooling2D()(x)
        f *= 2

    # Bottleneck
    x = Conv2D(f, 3, activation='relu', padding='same')(x)
    x = Conv2D(f, 3, activation='relu', padding='same')(x)
    f //= 2
    x = Conv2DTranspose(f, 2, strides=(2,2), padding='same')(x)

    # Upsampling
    for i in reversed(range(6)):
        x = Concatenate()([x, layers[i]])
        x = Conv2D(f, 3, activation='relu', padding='same')(x)
        x = Conv2D(f, 3, activation='relu', padding='same')(x)
        f //= 2
        if i > 0:
            x = Conv2DTranspose(f, 2, strides=(2,2), padding='same')(x)

    # Classification
    outputs = Conv2D(1, 1, activation='sigmoid')(x)

    model = Model(inputs, outputs)
    model.compile(optimizer=Adam(learning_rate=1e-4),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    return model

# Instantiate Model
model = unet()
model.summary()

In [ ]:
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')

model.fit(train_generator, validation_data=val_generator, epochs=10)